# Etape 3.3 : Detection d'anomalies

**Livrables** :
- Ce notebook `08_detection_anomalies.ipynb`
- Liste des anomalies `output/08_anomalies_detectees.csv`
- Rapport de recommandations d'audit

---
---

## Import

In [1]:
import sys
import os
from pathlib import Path
import psutil
import time
from datetime import datetime
import pandas as pd

---

## (optionnel) Enregistrement de la date de la dernière execution de ce notebook

In [2]:
print(f"- Date de la dernière execution de ce notebook : {datetime.now().strftime('%d/%m/%Y %H:%M:%S')} (FR)")

- Date de la dernière execution de ce notebook : 20/02/2026 20:33:20 (FR)


---

## (Optionnel) Mesure du temps de traitement global pour ce script - enregistrement de l'heure de début + estimation instantanée des ressources machine libres

In [3]:
## Heure de début
start_time_08 = time.time()

## Machine: current available RAM (in GB)
ram_available_08 = psutil.virtual_memory().available / (1024**3)

## Machine: current available CPU
logical = psutil.cpu_count()
physical = psutil.cpu_count(logical=False) or logical

cpu_used = psutil.cpu_percent(interval=2)
cpu_available_pct_08 = 100 - cpu_used

available_logical_08 = logical * cpu_available_pct_08 / 100
available_physical_08 = physical * cpu_available_pct_08 / 100

## Show available resources
print(f"- Current machine RAM available : {ram_available_08:.2f} GB")
print(f"- Current machine CPU available : {cpu_available_pct_08:.2f}%")
print(f"    Approx logical cores free  : {available_logical_08:.1f}")
print(f"    Approx physical cores free : {available_physical_08:.1f}")

- Current machine RAM available : 10.76 GB
- Current machine CPU available : 90.80%
    Approx logical cores free  : 14.5
    Approx physical cores free : 7.3


---

## Chemins des données

In [4]:
# ==============================================================================================================
#                                                   OUTPUTS
# ==============================================================================================================
OUT_DIR = (Path.cwd() / ".." / "output").resolve()
OUT_ANOMALIES_CSV = os.path.join(OUT_DIR, "08_anomalies_detectees.csv")
OUT_REC_AUDIT_MD = os.path.join(OUT_DIR, "08_rapport_audit.md") 

# ==============================================================================================================
#                                                   INPUTS
# ==============================================================================================================
IN_DIR = (Path.cwd() / ".." / "data").resolve()
IN_CONSO_ENRICHIE_CSV =  os.path.join(OUT_DIR, "05_consommations_enrichies.csv")

# ==============================================================================================================
#                                                    OTHERS
# ==============================================================================================================
TMP_DIR = (Path.cwd() / ".." / "my_tmp").resolve()
TMP_FILE_CSV = TMP_DIR / "tmp_08_resources.txt" # Enregistrer les metrics pour ce script

---

## Chargement des données

In [5]:
df_conso = pd.read_csv(IN_CONSO_ENRICHIE_CSV)

## Affichage de quelques infos
print("df_conso :")
print(f"    - Shape: {df_conso.shape}")
print(f"    - Colonnes: {df_conso.columns.tolist()}")
print()
# Info sur les types
print("    - Infos sur les types : ")
df_conso.info()
print()
## Appercu des donnees
print("    - Appercu des donnees : ")
df_conso.head()

df_conso :
    - Shape: (991, 31)
    - Colonnes: ['batiment_id', 'heure', 'consommation_moyenne', 'unite', 'date', 'type_energie', 'ts_h', 'nom', 'type', 'commune', 'surface_m2', 'annee_construction', 'classe_energetique', 'nb_occupants_moyen', 'temperature_c', 'humidite_pct', 'rayonnement_solaire_wm2', 'vitesse_vent_kmh', 'precipitation_mm', 'jour', 'mois', 'saison', 'jour_de_semaine', 'date_debut', 'date_fin', 'tarif_unitaire', 'cout_financier', 'conso_par_occupant', 'conso_par_m2', 'IPE', 'ecart_moyenne_categorie']

    - Infos sur les types : 
<class 'pandas.DataFrame'>
RangeIndex: 991 entries, 0 to 990
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   batiment_id              991 non-null    str    
 1   heure                    991 non-null    int64  
 2   consommation_moyenne     991 non-null    float64
 3   unite                    991 non-null    str    
 4   date                  

,batiment_id,heure,consommation_moyenne,unite,date,type_energie,ts_h,nom,type,commune,...,saison,jour_de_semaine,date_debut,date_fin,tarif_unitaire,cout_financier,conso_par_occupant,conso_par_m2,IPE,ecart_moyenne_categorie
0,BAT0043,22,280.88,m3,2023-01-01,eau,2023-01-01 22:00:00,Piscine Bordeaux 43,piscine,Bordeaux,...,Hiver,6.0,2023-01-01,2023-12-31,3.50,983.080,1.898,0.123,0.123,-401.254
1,BAT0100,17,4.52,m3,2023-01-01,eau,2023-01-01 17:00:00,Mediatheque Rennes 100,mediatheque,Rennes,...,Hiver,6.0,2023-01-01,2023-12-31,3.50,15.820,0.037,0.005,0.005,-92.701
2,BAT0006,3,5.50,kWh,2023-01-01,gaz,2023-01-01 03:00:00,Mairie Paris 6,mairie,Paris,...,Hiver,6.0,2023-01-01,2023-06-30,0.09,0.495,0.087,0.005,0.005,-43.286
3,BAT0126,10,126.88,kWh,2023-01-01,gaz,2023-01-01 10:00:00,Ecole Le Havre 126,ecole,Le Havre,...,Hiver,6.0,2023-01-01,2023-06-30,0.09,11.419,0.375,0.075,0.075,7.068
4,BAT0035,20,3.97,m3,2023-01-02,eau,2023-01-02 20:00:00,Mairie Toulouse 35,mairie,Toulouse,...,NaN,NaN,2023-01-01,2023-12-31,3.50,13.895,0.046,0.005,0.005,-44.816


---

## Identifier les pics de consommation anormaux (>3 ecarts-types)

In [6]:
df_conso["zscore_conso"] = (
    (df_conso["consommation_moyenne"]
     - df_conso.groupby("batiment_id")["consommation_moyenne"].transform("mean"))
    /
    df_conso.groupby("batiment_id")["consommation_moyenne"].transform("std")
)

pics_anormaux = df_conso[df_conso["zscore_conso"].abs() > 3]

pics_anormaux.head()
# => Détecte surconsommations inhabituelles.


,batiment_id,heure,consommation_moyenne,unite,date,type_energie,ts_h,nom,type,commune,...,jour_de_semaine,date_debut,date_fin,tarif_unitaire,cout_financier,conso_par_occupant,conso_par_m2,IPE,ecart_moyenne_categorie,zscore_conso


---

## Detecter les periodes de sous-consommation suspectes (batiment ferme non declare)

In [7]:
df_conso["zscore_bas"] = (
    (df_conso["consommation_moyenne"]
     - df_conso.groupby("batiment_id")["consommation_moyenne"].transform("mean"))
    /
    df_conso.groupby("batiment_id")["consommation_moyenne"].transform("std")
)

sous_conso = df_conso[df_conso["zscore_bas"] < -2]

sous_conso.head()


,batiment_id,heure,consommation_moyenne,unite,date,type_energie,ts_h,nom,type,commune,...,date_debut,date_fin,tarif_unitaire,cout_financier,conso_par_occupant,conso_par_m2,IPE,ecart_moyenne_categorie,zscore_conso,zscore_bas


---

## Reperer les batiments dont la consommation ne correspond pas a leur DPE

In [8]:
# --- On compare consommation réelle par m² vs référence théorique DPE
dpe_ref = {
    "A": 50, "B": 90, "C": 150, "D": 230,
    "E": 330, "F": 450, "G": 600
}

df_conso["conso_dpe_theorique"] = df_conso["classe_energetique"].map(dpe_ref)

df_conso["ecart_dpe"] = (
    df_conso["conso_par_m2"] - df_conso["conso_dpe_theorique"]
)

# --- Bâtiments incohérents
batiments_dpe_anormaux = (
    df_conso.groupby(["batiment_id","nom","type","commune","classe_energetique"])
    .agg(ecart_moyen_dpe=("ecart_dpe","mean"))
    .reset_index()
    .query("ecart_moyen_dpe > 50")
    .sort_values("ecart_moyen_dpe", ascending=False)
)

batiments_dpe_anormaux.head()

,batiment_id,nom,type,commune,classe_energetique,ecart_moyen_dpe


---

## Lister les batiments necessitant un audit energetique

In [9]:
audit = (
    df_conso.groupby(["batiment_id","nom","type","commune"])
    .agg(
        nb_pics=("zscore_conso", lambda x: (x.abs() > 3).sum()),
        conso_m2_moy=("conso_par_m2","mean"),
        cout_annuel=("cout_financier","mean"),
        ecart_dpe_moy=("ecart_dpe","mean")
    )
    .reset_index()
)

audit_prioritaire = audit[
    (audit["nb_pics"] > 5) |
    (audit["ecart_dpe_moy"] > 50) |
    (audit["conso_m2_moy"] > audit["conso_m2_moy"].quantile(0.9))
].sort_values(
    ["ecart_dpe_moy","nb_pics","conso_m2_moy"],
    ascending=False
)

audit_prioritaire.head()


,batiment_id,nom,type,commune,nb_pics,conso_m2_moy,cout_annuel,ecart_dpe_moy
117,BAT0118,Piscine Le Havre 118,piscine,Le Havre,0,0.277600,561.067800,-329.722400
118,BAT0119,Piscine Le Havre 119,piscine,Le Havre,0,0.256375,159.587625,-329.743625
91,BAT0092,Piscine Nice 92,piscine,Nice,0,0.552000,123.419000,-449.448000
145,BAT0146,Piscine Toulon 146,piscine,Toulon,0,0.350273,253.221000,-449.649727
135,BAT0136,Piscine Toulon 136,piscine,Toulon,0,0.334400,555.741400,-449.665600


---

## Liste des anomalies output/08_anomalies_detectees.csv

In [10]:
# --- Pics anormaux (>2 écarts-types recommandé) ---
df_conso["zscore_conso"] = (
    (df_conso["consommation_moyenne"]
     - df_conso.groupby("batiment_id")["consommation_moyenne"].transform("mean"))
    /
    df_conso.groupby("batiment_id")["consommation_moyenne"].transform("std")
)

anomalies_pics = df_conso[df_conso["zscore_conso"].abs() > 2]

# --- Sous-consommation suspecte ---
anomalies_sous_conso = df_conso[df_conso["zscore_conso"] < -2]

# --- Incohérence DPE ---
dpe_ref = {"A":50,"B":90,"C":150,"D":230,"E":330,"F":450,"G":600}
df_conso["conso_dpe_theorique"] = df_conso["classe_energetique"].map(dpe_ref)
df_conso["ecart_dpe"] = df_conso["conso_par_m2"] - df_conso["conso_dpe_theorique"]

anomalies_dpe = df_conso[df_conso["ecart_dpe"] > 50]

# --- Fusion des anomalies ---
anomalies = pd.concat([
    anomalies_pics.assign(type_anomalie="pic_conso"),
    anomalies_sous_conso.assign(type_anomalie="sous_conso"),
    anomalies_dpe.assign(type_anomalie="ecart_dpe")
]).drop_duplicates()

# --- Export CSV
anomalies.to_csv(OUT_ANOMALIES_CSV, index=False, encoding="utf-8-sig")

---

## Rapport de recommandations d'audit

In [11]:
rapport = """
# Rapport de recommandations d’audit énergétique

## Objectif

Ce rapport vise à identifier les bâtiments nécessitant une attention prioritaire en matière d’efficacité énergétique, à partir de l’analyse des consommations, des caractéristiques techniques et des conditions météorologiques. Les recommandations ci-dessous permettent de cibler les actions d’audit et d’optimisation.

---

## Constat général

L’analyse des données met en évidence plusieurs phénomènes :

* Des **pics de consommation ponctuels** suggérant des anomalies techniques ou des usages non maîtrisés.
* Des **périodes de sous-consommation** pouvant indiquer des fermetures temporaires, des travaux ou des problèmes de remontée de données.
* Des écarts entre la **performance énergétique théorique (DPE)** et la consommation réelle, révélant des inefficacités potentielles.
* Une influence modérée des variables météorologiques, notamment la température sur la consommation de chauffage.

---

## Bâtiments prioritaires pour audit

Les bâtiments suivants doivent être considérés comme prioritaires lorsqu’ils présentent un ou plusieurs critères :

* Consommation élevée rapportée à la surface (kWh/m²).
* Écart significatif avec la consommation théorique associée au DPE.
* Multiples anomalies de consommation (pics ou creux).
* Coût énergétique annuel élevé.

### Critères de priorisation

* Priorité haute : Surconsommation + anomalies répétées.
* Priorité moyenne : Écart DPE important ou forte variabilité.
* Priorité faible : Consommation cohérente avec les indicateurs.

---

## Recommandations techniques

### Systèmes énergétiques

* Vérifier les réglages de chauffage et la régulation thermique.
* Contrôler les installations électriques pour détecter les usages anormaux.
* Examiner les équipements à forte intensité énergétique (piscines, ventilation).

### Enveloppe du bâtiment

* Évaluer l’isolation thermique des bâtiments classés DPE E, F ou G.
* Identifier les pertes thermiques potentielles (fenêtres, toitures).

### Suivi et pilotage

* Déployer des alertes en cas de dépassement de seuil de consommation.
* Comparer régulièrement la performance réelle aux objectifs énergétiques.

---

## Actions rapides recommandées

* Ajuster les plages horaires de chauffage selon l’occupation réelle.
* Sensibiliser les occupants aux bonnes pratiques énergétiques.
* Vérifier les capteurs ou compteurs en cas de données incohérentes.

---

## Perspectives d’amélioration

* Mettre en place des tableaux de bord de suivi mensuel.
* Prioriser les rénovations énergétiques pour les bâtiments les plus énergivores.

---

## Conclusion

L’identification des anomalies et des écarts de performance constitue une base solide pour orienter les audits énergétiques. Une approche combinant analyse de données, expertise technique et suivi continu permettra d’améliorer durablement la performance énergétique du parc immobilier tout en réduisant les coûts d’exploitation.

"""

# Sauvegarder le rapport 
with open(f"{OUT_REC_AUDIT_MD}", 'w', encoding='utf-8') as f:
    f.write(rapport)

print(f"Rapport de synthese des insight sauvegardé : {OUT_REC_AUDIT_MD}")

Rapport de synthese des insight sauvegardé : C:\Users\joel\Downloads\ecf_energie\output\08_rapport_audit.md


---

## Libérer la mémoire (Optionnel) 

In [12]:
del df_conso

---

## (Optionnel) enregistrement dans un fichier temporaire du temps d'execution + ressources pour utilisation ultérieure (dans le script run_pipeline_hybride.py ou autres)

In [13]:
temps_execution_08 = time.time() - start_time_08
temps_resources = f"""
    Date : {datetime.now().strftime("%d/%m/%Y %H:%M:%S")} (FR)

    temps_exec_sec={temps_execution_08:.2f}
    ram_gb={ram_available_08:.2f}
    cpu_pct={cpu_available_pct_08:.2f}
    logi_cores={available_logical_08:.1f}
    physi_cores={available_physical_08:.1f}
"""

# Ecrire des données du temps d'execution + ressources dans le fichier TMP_FILE_CSV
TMP_FILE_CSV.write_text(temps_resources, encoding="utf-8")

137